># Content based filtering

Compared to collabarative filtering, the difference of content based filtering is that this uses the features of a given item for recommendation. Basically in this type of recommendation systems, normal ML classification and similarity techniques will be used.

For the implementation of such system, there are 3 main parts that we need to consider.

1. Content analyzer - Basically offline training of a model on the items. We may build vectors/profiles for each item which can be used later during recommendation process.

2. User Profiler -  We create profiles for users as well. This is to identify the unique person interests to match against the items of our system.

3. Item Retriever - The inference process where we match the user profile with items to get the recommendation. This is an online process.


>## Content Analyzer

Mainly based on what we called metadata. This include tags, descriptions, reviews etc. about an item. One of the important yet annoying part of this is recognizing what to use and what not to use. Because adding all will be expensive(computationally/development wise and actual cost) and adding less/ wrong features will yield in bad/weird recommendations.

This closely relates to NLP. Therefore many techniques in NLP world can e used such as Bag of Words, word2vec, TF-IDF, transformers, data cleaning steps(lematization, stop word removal). Some interesting pointers to related techniques are below.


- *Theres a python package named `stop-words` which include stop words from various languages. Install it using `pip install stop-words`.*

- *We can remove highest occuring and lowest occuring tokens from the dataset, if we are using a token based string vectorization method to transform strings.*

- *Can use stemmer/lemmatizer to reduce token forms in dataset as well. But again usefulness may depend on the application and the data you have.*



>### TF-IDF

- TF = Term Frequency (how many times the word appear in the document.)
- IDF = Inverse Document Frequency (Measure of how many documents have the considering word. If few documents have the considering word, higher the value would be.)

We take the log forms of above measures for calculations.

__<center>TF-IDF = TF(word, document)*IDF(word, documents)</center>__

<center>TF(word, document) = 1 + log(word frequency in the document)</center>
<center>IDF(word, documents) = log(Total number of documents) - log(num of documents with considering word)</center>




>### LDA (Latent Dirichlet Allocation)

In this ML technique words get allocated to hidden(latent) topics in the document distribution. Then those topics will be used to describe a document in a mathematical formula of percentages.


<center><image src="./images/LDA training.jpg" width="500px" /></center>

* Can use `pyLDAvis` package to visualize the LDA topics distribution. This helps to understand the proper topic number "k" we need to use for the algorithm.
* There are parameters `alpha` and `beta` in LDA which can be used to finetune the topic and word dstributions.
